In [ ]:
import socket
import sys
import pickle


class Client:
    """
    Client Class to perform lab 1. Connect to GCD to get a list of group members,
    send a HELLO to those members, and receive a message.
    """

    def __init__(self, hostname, port):
        """
        Constructor to initialize a Client object to talk to GCD.

        :param hostname: GCD hostname
        :param port: GCD port number
        """
        self.hostname = hostname
        self.port = port

    def connect_gcd(self):
        """
        Connects with GCD and gets the list of group members.

        :return: list of group members
        """
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as gcd_socket:
            try:
                # Connect to GCD server
                gcd_socket.connect((self.hostname, self.port))
                # Send 'BEGIN' message to GCD (as per the instructions)
                gcd_socket.sendall(pickle.dumps('BEGIN'))
                # Receive the response, unpickle it to get list of members
                response = gcd_socket.recv(4096) 
                members = pickle.loads(response)
                if isinstance(members, list):
                    return members
                else:
                    return None
            except Exception as e:
                print(f"Error connecting to GCD: {e}")
                sys.exit(1)

    def send_message_to_members(self, members):
        """
        Connects to group members, sends a HELLO message, and 
        displays the response.

        :param members: list of group members to connect to.
        """
        for member in members:
            host = member['host']
            port = member['port']
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as member_socket:
                member_socket.settimeout(1.5)  # Set 1500ms timeout
                try:
                    print(f'HELLO to Host: {host}, Port: {port}')
                    # Connect to the member
                    member_socket.connect((host, int(port)))
                    # Send 'HELLO' message
                    member_socket.sendall(pickle.dumps('HELLO'))
                    # Receive and unpickle the response
                    response = pickle.loads(member_socket.recv(4096))
                    print(f"Response from {host}:{port} - {response}")
                except socket.timeout:
                    print(f"Connection to {host}:{port} timed out.")
                except Exception as e:
                    print(f"Unable to connect to {host}:{port}: {e}")


if __name__ == '__main__':
    # Check that we have the correct number of arguments
    if len(sys.argv) != 3:
        print("Usage: python lab1.py GCD_HOST GCD_PORT")
        sys.exit(1)

    # Extract host and port from command line arguments
    gcd_host = sys.argv[1]
    gcd_port = int(sys.argv[2])

    # Initialize the client and connect to the GCD
    client = Client(gcd_host, gcd_port)
    group_members = client.connect_gcd()

    # If we received a valid list of members, send them the HELLO message
    if group_members:
        print("List of group members: \n", group_members)
        client.send_message_to_members(group_members)
    else:
        print("No group members received.")
